In [ ]:
%load_ext autoreload
%autoreload 2
%aimport utils

import altair as alt
import pandas as pd
import numpy as np
from vega_datasets import data

from constants import COLUMNS
from utils import apply_theme, get_visualization_subtitle
from web import for_website

In [ ]:
sites = pd.read_csv('../data/sites.csv', header=[0])

country_to_continent = {
    "France": "Europe",
    "Germany": "Europe",
    "Italy": "Europe",
    "Singapore": "Asia",
    "USA": "North America"
}

sites["Continent"] = sites["Country"].apply(lambda country: country_to_continent[country])

sites.head()

In [ ]:
def get_map_subtitle(continent):
    visualization_subtitle = get_visualization_subtitle(with_num_sites=False)
    continent_sites = sites.loc[sites["Continent"] == continent]
    num_continent_sites = continent_sites.shape[0]
    return f"{visualization_subtitle}  |  {num_continent_sites} Site" + ("s" if num_continent_sites != 1 else "")

In [ ]:
countries = alt.topo_feature(data.world_110m.url, 'countries')

COUNTRIES = ["France", "Germany", "Italy", "Singapore", "USA"]
COUNTRY_ID = [250, 276, 380, 702, 840]
COUNTRY_COLOR = ["#0072B2", "#E69F00", "#009E73", "#CC79A7", "#D55E00"]
COLOR_BY_COUNTRY = {COUNTRIES[i]: COUNTRY_COLOR[i] for i in range(len(COUNTRIES))} 
COLOR_BY_ID = {COUNTRY_ID[i]: COUNTRY_COLOR[i] for i in range(len(COUNTRY_ID))}
ID_COUNTRY_MAP = {COUNTRY_ID[i]: COUNTRIES[i] for i in range(len(COUNTRIES))}
color_scale = alt.Scale(domain=COUNTRIES, range=COUNTRY_COLOR)
color_id_scale = alt.Scale(domain=COUNTRY_ID, range=COUNTRY_COLOR)


background = alt.Chart(countries).mark_geoshape(
    fill="lightgray",
    stroke='white'
)

# city positions on background
locations = alt.Chart(sites).transform_aggregate(
    Latitude_Mean='mean(Latitude)',
    Longitude_Mean='mean(Longitude)',
    Hospitals_Total='sum(Hospitals)',
    Sites_Total='count()',
    groupby=['City','Country']
).mark_circle().encode(
    longitude='Longitude_Mean:Q',
    latitude='Latitude_Mean:Q',
    size=alt.Size( 'Hospitals_Total:Q', title='Number of Hospitals'),
    #size=alt.value(75),
    color=alt.Color('Country:N', scale=color_scale),
    tooltip=[alt.Tooltip('City:N', title="City"),alt.Tooltip('Sites_Total:Q', title="Sites Total"), alt.Tooltip('Hospitals_Total:Q', title="Hospitals Total")]
)

labels = locations.mark_text(
    align='center',
    baseline='top',
    dy=9
).encode(
    text='City',
    size=alt.value(10),
    color=alt.value('black')
)

In [ ]:
def get_north_america_map(c0=0, c1=0):
    north_america_map = ( background + locations + labels ).project(
        type= 'mercator',
        scale= 280,                          # Magnify
        center= [-95 + c0, 40 + c1],                     # [lon, lat]
        clipExtent= [[0, 0], [400, 300]],    # [[left, top], [right, bottom]]
    ).properties(
        title={
            "text": 'Sites in North America',
            "subtitle": get_map_subtitle("North America"),
            "subtitleColor": "gray"
        },
        width=300, height=308,
    )
    return north_america_map

north_america_map = get_north_america_map()
apply_theme(north_america_map, legend_orient="right")

In [ ]:
def get_europe_map(c0=0, c1=0):
    europe_map = ( background + locations + labels ).project(
        type='mercator',
        scale=800,                          # Magnify
        center=[7 + c0, 47 + c1],                     # [lon, lat]
        clipExtent=[[0, 0], [400, 300]],    # [[left, top], [right, bottom]]
    ).properties(
        title={
            "text": 'Sites in Europe',
            "subtitle": get_map_subtitle("Europe"),
            "subtitleColor": "gray"
        },
        width=300, height=308
    )
    return europe_map

europe_map = get_europe_map()
apply_theme(europe_map, legend_orient="bottom")

In [ ]:
def get_asia_map(c0=0, c1=0):
    asia_map = ( background + locations + labels ).project(
        type= 'mercator',
        scale= 400,                          # Magnify
        center= [100 + c0, 0 + c1],                     # [lon, lat]
        clipExtent= [[0, 0], [400, 300]],    # [[left, top], [right, bottom]]
    ).properties(
        title={
            "text": 'Sites in Asia',
            "subtitle": get_map_subtitle("Asia"),
            "subtitleColor": "gray"
        },
        width=300, height=308
    )
    return asia_map

asia_map = get_asia_map()
apply_theme(asia_map, legend_orient="right")

In [ ]:
continents_plot = apply_theme(alt.hconcat(
    get_north_america_map(-32, 15), 
    get_europe_map(-14, 7), 
    get_asia_map(-20, 20)
)).configure_concat(
    spacing=10
).configure_legend(
    titleFontSize=15,
    titleFontWeight=400,
    labelFontSize=14,
    labelFontWeight=300,
    padding=10,
    cornerRadius=0,
    orient="bottom",
    fillColor="white",
    strokeColor="lightgray",
    symbolType="circle",
    titleOrient="top",
    clipHeight=25
)

for_website(continents_plot, "Map", "Sites by Continent")

continents_plot